## Overview of the Jupyter Notebook

### Key Components:
- **Libraries Installation**: The notebook begins by installing necessary Python libraries such as `gradio`, `openai`, and `tiktoken` which are used for creating web interfaces, interacting with AI models, and token analysis, respectively.
- **Function Definitions**: Several Python functions are defined to perform tasks such as database backup, restore operations, and interaction logging. These functions help in managing data effectively.
- **Database Management**: Uses SQLite database for storing and managing interaction logs and templates.
- **Gradio Interface**: A web-based interface is set up using Gradio, enabling users to interact with the notebook through a web page, making it user-friendly for non-programmers.



In [1]:
!pip install -q dataset ipywidgets gradio openai tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
poetry 1.7.1 requires platformdirs<4.0.0,>=3.0.0, but you have platformdirs 4.2.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [2]:

import dataset
from datetime import datetime
import gradio as gr
from openai import OpenAI
#from google.colab import userdata
import tiktoken
import os
import shutil
import csv
import sqlite3
from datetime import datetime
import re

In [36]:
#setup global database variable
db = dataset.connect('sqlite:///interactions.db')
working_dir = '.'
backup_dir = './drive/MyDrive/InteractionTrackerBackups/'

In [37]:


def backup_db_files(source_dir, destination_dir):
    messages = []

    if not os.path.exists(source_dir):
        messages.append("Source directory does not exist.")
        return messages

    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
        messages.append(f"Destination directory created: {destination_dir}")

    db_extensions = ('.db', '.db-shm', '.db-wal')
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")  # Format datetime


    for filename in os.listdir(source_dir):
        if filename.endswith(db_extensions):
            source_file = os.path.join(source_dir, filename)
            modified_filename = f"{os.path.splitext(filename)[0]}_{current_time}{os.path.splitext(filename)[1]}"
            destination_file = os.path.join(destination_dir, modified_filename)
            shutil.copy2(source_file, destination_file)
            messages.append(f"Copied {filename} to {destination_file}")

    return messages

def backup_db_to_drive_as_csv(source_dir, destination_dir):
    messages = []

    if not os.path.exists(source_dir):
        messages.append("Source directory does not exist.")
        return messages

    db_extensions = ('.db', '.db-shm', '.db-wal')
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")  # Format datetime

    for filename in os.listdir(source_dir):
        if filename.endswith('.db'):  # Only process .db files for CSV conversion
            source_file = os.path.join(source_dir, filename)
            # Connect to the SQLite database
            conn = sqlite3.connect(source_file)
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()

            for table_name in tables:
                table_name = table_name[0]
                query = f"SELECT * FROM {table_name}"
                data = cursor.execute(query)

                csv_filename = f"{os.path.splitext(filename)[0]}_{table_name}_{current_time}.csv"
                csv_file_path = os.path.join(source_dir, csv_filename)

                with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
                    csv_writer = csv.writer(csv_file)
                    # Write headers
                    csv_writer.writerow([i[0] for i in cursor.description])
                    # Write data
                    csv_writer.writerows(data)

                messages.append(f"CSV file created in source directory: {csv_filename}")

                # If destination directory exists, copy the CSV file there
                if os.path.exists(destination_dir):
                    destination_csv_file_path = os.path.join(destination_dir, csv_filename)
                    shutil.copy2(csv_file_path, destination_csv_file_path)
                    messages.append(f"Copied {csv_filename} to {destination_dir}")

            conn.close()

    return messages


def autobackup_db_files(source_dir, destination_dir):
    messages = []

    if not os.path.exists(source_dir):
        messages.append("Source directory does not exist.")
        return messages

    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
        messages.append(f"Destination directory created: {destination_dir}")

    db_extensions = ('.db', '.db-shm', '.db-wal')
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")  # Format datetime


    for filename in os.listdir(source_dir):
        if filename.endswith(db_extensions):
            source_file = os.path.join(source_dir, filename)
            modified_filename = f"{os.path.splitext(filename)[0]}_autobackup{os.path.splitext(filename)[1]}"
            destination_file = os.path.join(destination_dir, modified_filename)
            shutil.copy2(source_file, destination_file)
            messages.append(f"Copied {filename} to {destination_file}")

    return messages

#create function for restoring the autobackup files
def restore_auto_backup(backup_dir, restore_dir, base_filename):
    messages = []

    if not os.path.exists(restore_dir):
        os.makedirs(restore_dir)
        messages.append(f"Restore directory created: {restore_dir}")

    autobackup_files = ["interactions_autobackup.db", "interactions_autobackup.db-shm", "interactions_autobackup.db-wal"]

    #copy the autobackup files to the restore directory but remove the _autobackup
    for filename in autobackup_files:
        source_file = os.path.join(backup_dir, filename)
        modified_filename = filename.replace("_autobackup", "")
        restore_file = os.path.join(restore_dir, modified_filename)
        shutil.copy2(source_file, restore_file)
        messages.append(f"Copied {filename} to {restore_file}")

    template_names = get_template_names()

    return messages, gr.Dropdown(label="Select Template", choices=template_names, value=template_names[0] if template_names else None)


def restore_db_files(backup_dir, restore_dir, base_filename):
    messages = []

    if not os.path.exists(restore_dir):
        os.makedirs(restore_dir)
        messages.append(f"Restore directory created: {restore_dir}")

    latest_common_timestamp = None
    candidate_files = {'.db': None, '.db-shm': None, '.db-wal': None}

    # Gather all files and their timestamps
    file_timestamps = {}
    for filename in os.listdir(backup_dir):
        if filename.startswith(base_filename) and any(filename.endswith(ext) for ext in ['.db', '.db-shm', '.db-wal']):
            match = re.search(r"_(\d{14})\.", filename)
            if match:
                timestamp = match.group(1)
                ext = os.path.splitext(filename)[1]
                if timestamp not in file_timestamps:
                    file_timestamps[timestamp] = {}
                file_timestamps[timestamp][ext] = filename

    # Determine the latest timestamp where all file types are present
    for timestamp, files in file_timestamps.items():
        if all(ext in files for ext in ['.db', '.db-shm', '.db-wal']):
            if not latest_common_timestamp or timestamp > latest_common_timestamp:
                latest_common_timestamp = timestamp
                candidate_files = files

    if latest_common_timestamp:
        # Restore the files with the latest common timestamp
        for ext, filename in candidate_files.items():
            source_file = os.path.join(backup_dir, filename)
            restore_file = os.path.join(restore_dir, f"{base_filename}{ext}")
            shutil.copy2(source_file, restore_file)
            messages.append(f"Restored {filename} to {restore_file}")
    else:
        messages.append("No complete set of backup files found with a common timestamp.")

    template_names = get_template_names()

    return messages, gr.Dropdown(label="Select Template", choices=template_names, value=template_names[0] if template_names else None)




In [38]:

# Instantiate the OpenAI client
def generate_ai_response(openai_api_key, model, temperature, max_tokens, user_input):
    client = OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": user_input}],
        max_tokens=max_tokens,
        temperature=temperature
    )
    return response.choices[0].message.content



def setup_database():
    global db
    if 'interactions' not in db.tables:
        db.create_table('interactions')
        db['interactions'].create_column('timestamp', db.types.string)
        db['interactions'].create_column('introduction', db.types.string)
        db['interactions'].create_column('field_1_intro', db.types.string)
        db['interactions'].create_column('field_1_name', db.types.string)
        db['interactions'].create_column('field_1_content', db.types.string)
        db['interactions'].create_column('field_2_intro', db.types.string)
        db['interactions'].create_column('field_2_name', db.types.string)
        db['interactions'].create_column('field_2_content', db.types.string)
        db['interactions'].create_column('field_3_intro', db.types.string)
        db['interactions'].create_column('field_3_name', db.types.string)
        db['interactions'].create_column('field_3_content', db.types.string)
        db['interactions'].create_column('field_4_intro', db.types.string)
        db['interactions'].create_column('field_4_name', db.types.string)
        db['interactions'].create_column('field_4_content', db.types.string)
        db['interactions'].create_column('conclusion', db.types.string)
        db['interactions'].create_column('generated_text', db.types.string)
    if 'templates' not in db.tables:
        db.create_table('templates')
        db['templates'].create_column('template_name', db.types.string)
        db['templates'].create_column('introduction', db.types.string)
        db['templates'].create_column('field_1_intro', db.types.string)
        db['templates'].create_column('field_1_name', db.types.string)
        db['templates'].create_column('field_1_content', db.types.string)
        db['templates'].create_column('field_2_intro', db.types.string)
        db['templates'].create_column('field_2_name', db.types.string)
        db['templates'].create_column('field_2_content', db.types.string)
        db['templates'].create_column('field_3_intro', db.types.string)
        db['templates'].create_column('field_3_name', db.types.string)
        db['templates'].create_column('field_3_content', db.types.string)
        db['templates'].create_column('field_4_intro', db.types.string)
        db['templates'].create_column('field_4_name', db.types.string)
        db['templates'].create_column('field_4_content', db.types.string)
        db['templates'].create_column('conclusion', db.types.string)

        default_template_exists = db['templates'].find_one(template_name='default')
        if not default_template_exists:
            db['templates'].insert({
                'template_name': 'default',
                'introduction': '',
                'field_1_intro': '',
                'field_1_name': '',
                'field_1_content': '',
                'field_2_intro': '',
                'field_2_name': '',
                'field_2_content': '',
                'field_3_intro': '',
                'field_3_name': '',
                'field_3_content': '',
                'field_4_intro': '',
                'field_4_name': '',
                'field_4_content': '',
                'conclusion': ''
            })

def log_interaction(introduction, field_1_intro, field_1_name, field_1_content, field_2_intro, field_2_name, field_2_content, field_3_intro, field_3_name, field_3_content, field_4_intro, field_4_name, field_4_content, conclusion, generated_text):
    global db
    table = db['interactions']
    table.insert({
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'introduction': introduction,
        'field_1_intro': field_1_intro,
        'field_1_name': field_1_name,
        'field_1_content': field_1_content,
        'field_2_intro': field_2_intro,
        'field_2_name': field_2_name,
        'field_2_content': field_2_content,
        'field_3_intro': field_3_intro,
        'field_3_name': field_3_name,
        'field_3_content': field_3_content,
        'field_4_intro': field_4_intro,
        'field_4_name': field_4_name,
        'field_4_content': field_4_content,
        'conclusion': conclusion,
        'generated_text': generated_text
    })

def save_template(template_name, introduction, field_1_intro, field_1_name, field_1_content, field_2_intro, field_2_name, field_2_content, field_3_intro, field_3_name, field_3_content, field_4_intro, field_4_name, field_4_content, conclusion):
    global db
    table = db['templates']
    table.insert({
        'template_name': template_name,
        'introduction': introduction,
        'field_1_intro': field_1_intro,
        'field_1_name': field_1_name,
        'field_1_content': field_1_content,
        'field_2_intro': field_2_intro,
        'field_2_name': field_2_name,
        'field_2_content': field_2_content,
        'field_3_intro': field_3_intro,
        'field_3_name': field_3_name,
        'field_3_content': field_3_content,
        'field_4_intro': field_4_intro,
        'field_4_name': field_4_name,
        'field_4_content': field_4_content,
        'conclusion': conclusion
    })

def read_interactions():
    global db
    interactions = db['interactions']
    interaction_ids = ""

    for interaction in interactions.all(order_by='-timestamp'):
        #add interaction timestamp and all of the field names
        interaction_ids += (f"{interaction['timestamp']} - {interaction['field_1_name']} - {interaction['field_2_name']} - {interaction['field_3_name']} - {interaction['field_4_name']}\n")

    return interaction_ids

def load_interaction_details(timestamp):
    global db
    #trim timestamp
    timestamp = timestamp.strip()
    interaction = db['interactions'].find_one(timestamp=timestamp)
    if interaction:
        return (
            interaction['introduction'], interaction['field_1_intro'], interaction['field_1_name'], interaction['field_1_content'],
            interaction['field_2_intro'], interaction['field_2_name'], interaction['field_2_content'],
            interaction['field_3_intro'], interaction['field_3_name'], interaction['field_3_content'],
            interaction['field_4_intro'], interaction['field_4_name'], interaction['field_4_content'],
            interaction['conclusion'], interaction['generated_text']
        )
    else:
        return ("", "", "", "", "", "", "", "", "", "", "", "", "", "" , "")

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def calculate_tokens_and_cost(model, user_input, encoding_name, max_tokens):
    num_tokens = num_tokens_from_string(user_input, "cl100k_base")

    # Model pricing structure
    pricing = {
        'gpt-4o': {'input': 5, 'output': 15},
        'gpt-4-turbo': {'input': 10, 'output': 30},
        'gpt-3.5-turbo': {'input': 0.50, 'output': 1.50}
    }

    # Calculate input and output token costs based on the model's pricing
    input_cost = (num_tokens / 1_000_000) * pricing[model]['input']
    output_cost = (max_tokens / 1_000_000) * pricing[model]['output']

    # Total cost calculation
    total_cost = input_cost + output_cost

    return num_tokens, total_cost

def combine_fields_and_estimate_costs(model, temperature, max_tokens, introduction, field_1_intro, field_1_name, field_1_content, field_2_intro, field_2_name, field_2_content, field_3_intro, field_3_name, field_3_content, field_4_intro, field_4_name, field_4_content, conclusion):

    consolidated_info = (
        f"Introduction: {introduction}\n"
        f"{field_1_intro}: {field_1_name} - {field_1_content}\n"
        f"{field_2_intro}: {field_2_name} - {field_2_content}\n"
        f"{field_3_intro}: {field_3_name} - {field_3_content}\n"
        f"{field_4_intro}: {field_4_name} - {field_4_content}\n"
        f"Conclusion: {conclusion}"
    )

    num_tokens, total_cost = calculate_tokens_and_cost(model, consolidated_info, "cl100k_base", max_tokens)

    message = f"Number of Tokens: {num_tokens}\nTotal Cost: ${total_cost:.2f}"
    return message

def log_and_display_interactions(openai_api_key, autobackup_checkbox, template_name, model, temperature, max_tokens, introduction, field_1_intro, field_1_name, field_1_content, field_2_intro, field_2_name, field_2_content, field_3_intro, field_3_name, field_3_content, field_4_intro, field_4_name, field_4_content, conclusion):
    setup_database()
    consolidated_info = (
        f"Introduction: {introduction}\n"
        f"{field_1_intro}: {field_1_name} - {field_1_content}\n"
        f"{field_2_intro}: {field_2_name} - {field_2_content}\n"
        f"{field_3_intro}: {field_3_name} - {field_3_content}\n"
        f"{field_4_intro}: {field_4_name} - {field_4_content}\n"
        f"Conclusion: {conclusion}"
    )
    generated_text = generate_ai_response(openai_api_key, model, temperature, max_tokens, consolidated_info)

    log_interaction(introduction, field_1_intro, field_1_name, field_1_content, field_2_intro, field_2_name, field_2_content, field_3_intro, field_3_name, field_3_content, field_4_intro, field_4_name, field_4_content, conclusion, generated_text)

    logged_interactions = read_interactions()

    if autobackup_checkbox:
        autobackup_db_files(working_dir, backup_dir)

    return generated_text

def save_template_handler(autobackup_checkbox, template_name, introduction, field_1_intro, field_1_name, field_1_content, field_2_intro, field_2_name, field_2_content, field_3_intro, field_3_name, field_3_content, field_4_intro, field_4_name, field_4_content, conclusion):



    if not template_name:
        template_name = f"{field_1_name}_{field_2_name}_{field_3_name}_{field_4_name}"

    #check if template name is already in database and return error message if it is
    template_exists = db['templates'].find_one(template_name=template_name)
    if template_exists:
        return f"Template '{template_name}' already exists. Please choose a different name.", gr.Dropdown(label="Select Template", choices=template_names, value=template_name)

    save_template(template_name, introduction, field_1_intro, field_1_name, field_1_content, field_2_intro, field_2_name, field_2_content, field_3_intro, field_3_name, field_3_content, field_4_intro, field_4_name, field_4_content, conclusion)
    template_names = get_template_names()

    if autobackup_checkbox:
        autobackup_db_files(working_dir, backup_dir)

    return f"Template '{template_name}' saved successfully.", gr.Dropdown(label="Select Template", choices=template_names, value=template_name)

def delete_template_handler(autobackup_checkbox, template_name):
    global db
    db['templates'].delete(template_name=template_name)
    template_names = get_template_names()

    if autobackup_checkbox:
        autobackup_db_files(working_dir, backup_dir)

    return f"Template '{template_name}' deleted successfully.", gr.Dropdown(label="Select Template", choices=template_names, value=template_names[0] if template_names else None)

def get_template_names():
    global db
    templates = db['templates']
    return [template['template_name'] for template in templates.all()]

def load_template_details(template_name):
    global db
    template = db['templates'].find_one(template_name=template_name)
    if template:
        return (template['introduction'], template['field_1_intro'], template['field_1_name'], template['field_1_content'],
                template['field_2_intro'], template['field_2_name'], template['field_2_content'],
                template['field_3_intro'], template['field_3_name'], template['field_3_content'],
                template['field_4_intro'], template['field_4_name'], template['field_4_content'],
                template['conclusion'])
    else:
        return ("", "", "", "", "", "", "", "", "", "", "", "", "", "")

In [39]:
#Initial database setup
setup_database()

#Used in Gradio interface below
openai_model_list = ['gpt-3.5-turbo', 'gpt-4o', 'gpt-4-turbo']

In [42]:


# Gradio interface components
with gr.Blocks() as demo:
    with gr.Tab("Templates"):
        with gr.Row():
            template_selector = gr.Dropdown(label="Select Template", choices=get_template_names(), value=get_template_names()[0] if get_template_names() else None, allow_custom_value=True)
            load_template_button = gr.Button("Load Template", variant="primary")
            save_template_button = gr.Button("Save Template", variant="secondary")
            delete_template_button = gr.Button("Delete Template", variant="stop")
            estimate_costs_button = gr.Button("Estimate Costs", variant="primary")
        with gr.Row():
            template_name_input = gr.Textbox(placeholder='Enter template name (optional)', label='Template Name', scale=2)
            model_input = gr.Dropdown(label='Model', choices=openai_model_list, value=openai_model_list[0])
            temperature_input = gr.Slider(minimum=0, maximum=1, value=0.5, label='Temperature', scale=1)
            max_tokens_input = gr.Slider(minimum=10, maximum=4096, value=500, label='Max Tokens', scale=2)
            openai_api_key = gr.Textbox(placeholder='Enter OpenAI API Key', label='OpenAI API Key', type='password', scale=2)
        with gr.Row():
            backup_button = gr.Button("Backup Database", variant="secondary", scale=1)
            restore_button = gr.Button("Restore Database", variant="secondary", scale=1)
            autobackup_checkbox = gr.Checkbox(label='Autobackup', value=True)
            restore_autobackup_button = gr.Button("Restore Autobackup", variant="secondary", scale=1)

        #add status_area for return messages from buttons
        with gr.Row():
            status_area = gr.Textbox(placeholder='View status messages here', label='Status Message', lines=3, scale=2)
        with gr.Row():
            introduction_input = gr.Textbox(placeholder='Type the introduction...', label='Introduction', scale=1)
        with gr.Row():
            with gr.Column():
                field_1_intro_input = gr.Textbox(placeholder='Enter first field introduction...', label='First Field Introduction')
                field_1_name_input = gr.Textbox(placeholder='Enter first field name...', label='First Field Name')
                field_1_content_input = gr.Textbox(placeholder='Enter first field content...', label='First Field Content')
            with gr.Column():
                field_2_intro_input = gr.Textbox(placeholder='Enter second field introduction...', label='Second Field Introduction')
                field_2_name_input = gr.Textbox(placeholder='Enter second field name...', label='Second Field Name')
                field_2_content_input = gr.Textbox(placeholder='Enter second field content...', label='Second Field Content')
        with gr.Row():
            with gr.Column():
                field_3_intro_input = gr.Textbox(placeholder='Enter third field introduction...', label='Third Field Introduction')
                field_3_name_input = gr.Textbox(placeholder='Enter third field name...', label='Third Field Name')
                field_3_content_input = gr.Textbox(placeholder='Enter third field content...', label='Third Field Content')
            with gr.Column():
                field_4_intro_input = gr.Textbox(placeholder='Enter fourth field introduction...', label='Fourth Field Introduction')
                field_4_name_input = gr.Textbox(placeholder='Enter fourth field name...', label='Fourth Field Name')
                field_4_content_input = gr.Textbox(placeholder='Enter fourth field content...', label='Fourth Field Content')
        with gr.Row():
            conclusion_input = gr.Textbox(placeholder='Enter conclusion...', label='Conclusion', scale=1)
        with gr.Row():
            log_interaction_button = gr.Button("Send to LLM", variant="primary")
            save_prompt_button = gr.Button("Log without sending", variant="secondary")
            clear_button = gr.Button("Clear Fields", variant="stop")
            export_csv_button = gr.Button("Export CSV", variant="primary")
        with gr.Row():
            output_area = gr.Textbox(label='Interaction Log', lines=10, scale=2, show_copy_button=True)

        with gr.Row():
            interaction_to_load = gr.Textbox(label='Interaction Timestamp', scale=2)
            load_interaction_button = gr.Button("Load Interaction", variant="primary", scale=2)
            refresh_button = gr.Button("Refresh List", variant="secondary", scale=2)
        with gr.Row():
            interaction_list = gr.Textbox(label='Interaction List', lines=10, scale=2)

    # Event Listeners
    refresh_button.click(
        fn=lambda: read_interactions(),
        inputs=[],
        outputs=[interaction_list]
    )
    load_interaction_button.click(
            fn=load_interaction_details,
            inputs=[interaction_to_load],
            outputs=[introduction_input, field_1_intro_input, field_1_name_input, field_1_content_input,
                     field_2_intro_input, field_2_name_input, field_2_content_input,
                     field_3_intro_input, field_3_name_input, field_3_content_input,
                     field_4_intro_input, field_4_name_input, field_4_content_input, conclusion_input, output_area]
    )

    log_interaction_button.click(
        fn=log_and_display_interactions,
        inputs=[openai_api_key, autobackup_checkbox, template_name_input, model_input, temperature_input, max_tokens_input, introduction_input, field_1_intro_input, field_1_name_input, field_1_content_input, field_2_intro_input, field_2_name_input, field_2_content_input, field_3_intro_input, field_3_name_input, field_3_content_input, field_4_intro_input, field_4_name_input, field_4_content_input, conclusion_input],
        outputs=[output_area]
    )
    save_template_button.click(
        fn=save_template_handler,
        inputs=[autobackup_checkbox, template_name_input, introduction_input, field_1_intro_input, field_1_name_input, field_1_content_input, field_2_intro_input, field_2_name_input, field_2_content_input, field_3_intro_input, field_3_name_input, field_3_content_input, field_4_intro_input, field_4_name_input, field_4_content_input, conclusion_input],
        outputs=[status_area, template_selector]
    )
    clear_button.click(
        fn=lambda: ("", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", ""),
        inputs=[],
        outputs=[template_name_input, introduction_input, field_1_intro_input, field_1_name_input, field_1_content_input, field_2_intro_input, field_2_name_input, field_2_content_input, field_3_intro_input, field_3_name_input, field_3_content_input, field_4_intro_input, field_4_name_input, field_4_content_input, conclusion_input, output_area]
    )
    load_template_button.click(
        fn=load_template_details,
        inputs=[template_selector],
        outputs=[introduction_input, field_1_intro_input, field_1_name_input, field_1_content_input,
                 field_2_intro_input, field_2_name_input, field_2_content_input,
                 field_3_intro_input, field_3_name_input, field_3_content_input,
                 field_4_intro_input, field_4_name_input, field_4_content_input, conclusion_input]
    )
    backup_button.click(
        fn=lambda: backup_db_files(working_dir, backup_dir),
        inputs=[],
        outputs=[status_area]
    )
    restore_button.click(
        fn=lambda: restore_db_files(backup_dir, working_dir, 'interactions'),
        inputs=[],
        outputs=[status_area, template_selector]
    )
    restore_autobackup_button.click(
        fn=lambda: restore_auto_backup(backup_dir, working_dir, 'interactions'),
        inputs=[],
        outputs=[status_area, template_selector]
    )
    estimate_costs_button.click(
        fn=combine_fields_and_estimate_costs,
        inputs=[model_input, temperature_input, max_tokens_input, introduction_input, field_1_intro_input, field_1_name_input, field_1_content_input, field_2_intro_input, field_2_name_input, field_2_content_input, field_3_intro_input, field_3_name_input, field_3_content_input, field_4_intro_input, field_4_name_input, field_4_content_input, conclusion_input],
        outputs=[status_area]
    )
    export_csv_button.click(
        fn=lambda: backup_db_to_drive_as_csv(working_dir, backup_dir),
        inputs=[],
        outputs=[status_area]
    )
    delete_template_button.click(
        fn=delete_template_handler,
        inputs=[autobackup_checkbox, template_selector],
        outputs=[status_area, template_selector]
    )
    save_prompt_button.click(
        fn=log_interaction,
        inputs=[introduction_input, field_1_intro_input, field_1_name_input, field_1_content_input, field_2_intro_input, field_2_name_input, field_2_content_input, field_3_intro_input, field_3_name_input, field_3_content_input, field_4_intro_input, field_4_name_input, field_4_content_input, conclusion_input, output_area],
        outputs=[output_area]
    )



In [43]:
#launch the interface
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Documentation for All the Buttons:


1. **Refresh List Button:**
   - **Dataflow:** This button triggers the `read_interactions()` function, which reads data from the `interactions` table in the SQLite database and displays a list of interaction timestamps and details in the `interaction_list` textbox.
   - **Summary:** Refreshes and displays a list of all logged interactions.

2. **Load Interaction Button:**
   - **Dataflow:** Upon clicking, it takes the timestamp input from `interaction_to_load`, passes it to `load_interaction_details()`, and fills various text fields with details from the specified interaction record in the database.
   - **Summary:** Loads the details of a specific interaction based on its timestamp for review or editing.

3. **Log Interaction Button:**
   - **Dataflow:** Gathers data from input fields related to interaction details, sends them to the `log_and_display_interactions()` function, which logs these details into the database and optionally triggers an automatic backup. It also makes a call to the OpenAI API for text generation.
   - **Summary:** Logs details of an interaction and optionally backs up the database.

4. **Save Template Button:**
   - **Dataflow:** Collects data from template-related input fields, uses `save_template_handler()` to save these details as a new template in the database, and optionally performs an automatic backup.
   - **Summary:** Saves the current form data as a reusable template and optionally backs up the data.

5. **Clear Fields Button:**
   - **Dataflow:** Resets all input fields in the form to their default empty state.
   - **Summary:** Clears all the input fields for a fresh start on entering new data.

6. **Load Template Button:**
   - **Dataflow:** Takes the selected template name from the dropdown, retrieves its details from the database using `load_template_details()`, and populates the input fields with these details.
   - **Summary:** Loads a selected template's details into the form for quick use or modification.

7. **Backup Database Button:**
   - **Dataflow:** Initiates the `backup_db_files()` function to copy the current state of the database files to a backup directory.
   - **Summary:** Creates a backup of the current database state for recovery purposes.

8. **Restore Database Button:**
   - **Dataflow:** Uses `restore_db_files()` to replace the current database files with those from the most recent backup that has all necessary components (.db, .db-shm, .db-wal).
   - **Summary:** Restores the database to its most recent complete backup state.

9. **Restore Autobackup Button:**
   - **Dataflow:** Invokes `restore_auto_backup()` to specifically restore files from an automatic backup process, ensuring the database is reverted to the last automatically saved state.
   - **Summary:** Restores the database using the last set of automatically backed-up files.

10. **Estimate Costs Button:**
    - **Dataflow:** Aggregates the input from various fields to form a comprehensive description of an interaction, which is then used to calculate the cost of processing this text through the OpenAI API using `combine_fields_and_estimate_costs()`.
    - **Summary:** Estimates the cost of generating responses using the specified AI model based on the input data.

11. **Export CSV Button:**
    - **Dataflow:** Converts the contents of the SQLite database into CSV files and saves them to a designated backup directory through the `backup_db_to_drive_as_csv()` function.
    - **Summary:** Exports the database contents to CSV files for external use or analysis.
